In [31]:
import requests
import csv

In [32]:
def fetch_closed_issues(repo_link):
    # Extracting the owner and repo name from the provided GitHub repository link
    owner, repo = repo_link.split('/')[-2:]
    api_url = f"https://api.github.com/repos/{owner}/{repo}/issues"

    # Parameters to fetch closed issues
    params = {
        "state": "closed",
        "per_page": 100  # Fetching 100 items per page, you can adjust this number
    }
    print(api_url)

    issues = []

    # Fetching closed issues from the GitHub API using pagination
    while True:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            issues += response.json()
            # Check if there are more pages
            if 'next' in response.links:
                api_url = response.links['next']['url']
            else:
                break
        else:
            print("Failed to fetch closed issues:", response.status_code)
            return None

    return issues

In [33]:
def save_to_csv(issues, csv_filename):
    # Extracting relevant information from each issue
    data = []
    for issue in issues:
        # Only include the issue if the body is not empty
        if issue["body"] is not None:
            data.append({
                "Issue ID": issue["number"],
                "Title": issue["title"],
                "Issue Body": issue["body"],
                "Closed At": issue["closed_at"]
            })

    # Writing data to CSV file
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Issue ID", "Title", "Issue Body", "Closed At"])
        writer.writeheader()
        writer.writerows(data)

In [34]:

repo_link = "https://github.com/umami-software/umami"
issues = fetch_closed_issues(repo_link)

if issues:
    save_to_csv(issues, "crawled_issue/data.csv")
    print("Closed issues data saved to data.csv")
else:
    print("No closed issues found or failed to fetch closed issues.")

https://api.github.com/repos/umami-software/umami/issues
Failed to fetch closed issues: 403
No closed issues found or failed to fetch closed issues.
